In [ ]:
# 安裝必要套件
!pip install langgraph openai


In [ ]:
# 導入模組
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from openai import OpenAI
from typing import Dict
from google.colab import userdata

# 定義狀態結構
@dataclass
class SummaryState:
    summary: str = ""  # 生成的摘要
    post: str = ""  # 生成的X貼文
    feedback: str = ""  # 風格反饋
    iteration: int = 0  # 迭代次數
    input: str = ""  # 可選的新聞輸入

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 從Colab環境獲取金鑰

# 語言模型調用函數
def llm_call(prompt: str, max_tokens: int = 150) -> str:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.5  # 保持中立語氣
    )
    return response.choices[0].message.content.strip()

# 貼文生成節點
def post_node(state: SummaryState) -> Dict:
    prompt = f"將以下100字摘要改編為50~100字X貼文，語氣中立，簡潔吸睛，包含2~3個標籤（如#科技）：\n{state.summary}"
    state.post = llm_call(prompt, max_tokens=100)
    return {"post": state.post}

# 風格檢查節點
def style_node(state: SummaryState) -> Dict:
    prompt = f"檢查以下X貼文是否語氣中立、標籤適切（2~3個，符合主題）。若語氣偏差或標籤不佳，提供修正建議；若無，回覆‘無問題’：\n貼文：{state.post}"
    state.feedback = llm_call(prompt)
    state.iteration += 1
    return {"feedback": state.feedback, "iteration": state.iteration}

# 構建工作流程
workflow = StateGraph(SummaryState)
workflow.add_node("post_node", post_node)
workflow.add_node("style_node", style_node)
workflow.set_entry_point("post_node")
workflow.add_edge("post_node", "style_node")
workflow.add_conditional_edges(
    "style_node",
    lambda state: "end" if "無問題" in state.feedback or state.iteration >= 2 else "post_node",
    {"post_node": "post_node", "end": END}
)
graph = workflow.compile()

# 範例摘要（假設已從文章2生成）
summary = """
人工智慧技術將醫療影像診斷準確率提升至90%，遠超傳統方法的70%。這項突破利用深度學習與大數據分析，可降低癌症篩查誤診率30%，並減輕醫生負擔。未來有望推動智慧醫療發展，但仍需解決資料隱私等挑戰。
"""

# 執行貼文風格優化
result = graph.invoke(SummaryState(summary=summary))
print(f"優化後的X貼文：\n{result['post']}")
print(f"風格反饋：\n{result['feedback']}")